In [18]:
# import dependencies
import pandas as pd
import numpy as np
import timeit
import matplotlib.pyplot as plt
# Import the requests library.
import requests
# Import the API key.
from config import weather_api_key
# Import the datetime module from the datetime library.
from datetime import datetime
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Import the time module.
import time
# Import linear regression from the SciPy stats module.
from scipy.stats import linregress

In [19]:
# Step 2: create a list of latitudes and Longitudes and zip them
lat=np.random.uniform(low=-90.000,high=90.000, size=1500)
lngs=np.random.uniform(low=-180.000,high=180.000,size=1500)
lat_lngs=zip(lat,lngs)
lat_lngs

In [20]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [21]:
coordinates

[(14.434263991485523, 106.67275899303598),
 (19.689281990293665, 0.8357800833541091),
 (-51.09333732756929, 65.6365176131701),
 (-87.29586515850986, -15.333825944551961),
 (-29.96980754814866, 11.751233366289426),
 (17.102876060349928, 22.089158153366753),
 (48.51742135397819, -168.65202831935517),
 (7.881096951088651, -107.50088195877059),
 (11.557214075129181, -168.48882317148107),
 (30.71558462401522, -13.82193241279495),
 (55.20264512324417, -93.82433978073709),
 (6.751269459934164, 113.83125173588581),
 (44.760215712657526, -7.751366856012197),
 (11.394308408623004, -36.71843437532405),
 (6.117524866994657, 11.762419520644471),
 (4.825873055693137, -45.97176430515063),
 (82.32603169793057, 75.15134174947707),
 (-8.542738829137079, 41.73721458860126),
 (51.127950098542925, 82.31295830186224),
 (19.76558361375197, -17.637813019473953),
 (27.573793305115814, 64.43814825582865),
 (59.1338604069762, -63.410130995661504),
 (14.407397334038194, 53.9491766146422),
 (-13.398701568783764, -

In [22]:
# Step3 : Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

601

In [23]:
cities

['champasak',
 'tessalit',
 'saint-philippe',
 'ushuaia',
 'luderitz',
 'biltine',
 'bethel',
 'lazaro cardenas',
 'kapaa',
 'teguise',
 'thompson',
 'labuan',
 'cervo',
 'sao filipe',
 'banyo',
 'amapa',
 'dikson',
 'kilindoni',
 'zmeinogorsk',
 'nouadhibou',
 'dalbandin',
 'iqaluit',
 'salalah',
 'cavalcante',
 'sao felix do xingu',
 'mar del plata',
 'attawapiskat',
 'makakilo city',
 'saint-louis',
 'albany',
 'olafsvik',
 'ust-tsilma',
 'barrow',
 'seoul',
 'shingu',
 'punta arenas',
 'puerto ayora',
 'butaritari',
 'mindelo',
 'hilo',
 'rikitea',
 'taolanaro',
 'catalina',
 'port alfred',
 'dongsheng',
 'jamestown',
 'qaanaaq',
 'olinda',
 'atar',
 'hauterive',
 'new norfolk',
 'atuona',
 'saskylakh',
 'san patricio',
 'bluff',
 'kushiro',
 'hermanus',
 'lasa',
 'yellowknife',
 'bredasdorp',
 'georgetown',
 'carnarvon',
 'havre-saint-pierre',
 'busselton',
 'souillac',
 'tazovskiy',
 'verkhnevilyuysk',
 'flinders',
 'ponta do sol',
 'lavrentiya',
 'nizhneyansk',
 'mataura',
 'kis

In [24]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)# Starting URL for Weather Map API Call.

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=e0321ca41a7cfb548ec58b2953f7ca66


In [25]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [34]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_Weather=city_weather["weather"][0]["description"]
        try:
            city_rain=city_weather["rain"]["3h"]
        except KeyError:
            city_rain=0
        try:
            city_snow=city_weather["snow"]["3h"]
        except KeyError:
            city_snow=0
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Current description":city_Weather,
                          "Rain inches(last 3 hrs)":city_rain,
                          "Snow inches(last 3 hrs)":city_snow})

# #If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 2 of Set 13 | champasak
Processing Record 3 of Set 13 | tessalit
Processing Record 4 of Set 13 | saint-philippe
Processing Record 5 of Set 13 | ushuaia
Processing Record 6 of Set 13 | luderitz
Processing Record 7 of Set 13 | biltine
Processing Record 8 of Set 13 | bethel
Processing Record 9 of Set 13 | lazaro cardenas
Processing Record 10 of Set 13 | kapaa
Processing Record 11 of Set 13 | teguise
Processing Record 12 of Set 13 | thompson
Processing Record 13 of Set 13 | labuan
Processing Record 14 of Set 13 | cervo
Processing Record 15 of Set 13 | sao filipe
Processing Record 16 of Set 13 | banyo
Processing Record 17 of Set 13 | amapa
Processing Record 18 of Set 13 | dikson
Processing Record 19 of Set 13 | kilindoni
Processing Record 20 of Set 13 | zmeinogorsk
Processing Record 21 of Set 13 | nouadhibou
Processing Record 22 of Set 13 | dalbandin
Processing Record 23 of Set 13 | iqaluit
Processing Record 24 of Set 13 | salalah
Processing Record 25 of Set 13 | cavalcant

Processing Record 34 of Set 16 | avarua
Processing Record 35 of Set 16 | yerofey pavlovich
Processing Record 36 of Set 16 | katsuura
Processing Record 37 of Set 16 | longyearbyen
Processing Record 38 of Set 16 | charters towers
Processing Record 39 of Set 16 | kaeo
Processing Record 40 of Set 16 | sur
Processing Record 41 of Set 16 | torbay
Processing Record 42 of Set 16 | gangotri
City not found. Skipping...
Processing Record 43 of Set 16 | hambantota
Processing Record 44 of Set 16 | angoche
Processing Record 45 of Set 16 | vila velha
Processing Record 46 of Set 16 | sambava
Processing Record 47 of Set 16 | saint-augustin
Processing Record 48 of Set 16 | santana do livramento
Processing Record 49 of Set 16 | quirinopolis
Processing Record 50 of Set 16 | sulangan
Processing Record 1 of Set 17 | mahibadhoo
Processing Record 2 of Set 17 | oyama
Processing Record 3 of Set 17 | tiarei
Processing Record 4 of Set 17 | severo-kurilsk
Processing Record 5 of Set 17 | samur
Processing Record 6 o

Processing Record 19 of Set 20 | bima
Processing Record 20 of Set 20 | borogontsy
Processing Record 21 of Set 20 | klaksvik
Processing Record 22 of Set 20 | san vicente
Processing Record 23 of Set 20 | picos
Processing Record 24 of Set 20 | matoury
Processing Record 25 of Set 20 | san carlos de bariloche
Processing Record 26 of Set 20 | nioki
Processing Record 27 of Set 20 | petropavlovskaya
Processing Record 28 of Set 20 | sierra vista
Processing Record 29 of Set 20 | gorontalo
Processing Record 30 of Set 20 | luganville
Processing Record 31 of Set 20 | vila franca do campo
Processing Record 32 of Set 20 | puqi
Processing Record 33 of Set 20 | skovorodino
Processing Record 34 of Set 20 | flin flon
Processing Record 35 of Set 20 | mogwase
Processing Record 36 of Set 20 | semnan
Processing Record 37 of Set 20 | boende
Processing Record 38 of Set 20 | puerto colombia
Processing Record 39 of Set 20 | vestmannaeyjar
Processing Record 40 of Set 20 | chongwe
Processing Record 41 of Set 20 | 

Processing Record 5 of Set 24 | bundaberg
Processing Record 6 of Set 24 | san borja
Processing Record 7 of Set 24 | kasongo
Processing Record 8 of Set 24 | kiunga
Processing Record 9 of Set 24 | geraldton
Processing Record 10 of Set 24 | hovd
Processing Record 11 of Set 24 | boueni
Processing Record 12 of Set 24 | never
Processing Record 13 of Set 24 | colquechaca
Processing Record 14 of Set 24 | arlit
Processing Record 15 of Set 24 | tymovskoye
Processing Record 16 of Set 24 | barra do garcas
Processing Record 17 of Set 24 | erdaojiang
Processing Record 18 of Set 24 | safford
Processing Record 19 of Set 24 | kuche
City not found. Skipping...
Processing Record 20 of Set 24 | ajdabiya
Processing Record 21 of Set 24 | sabratah
Processing Record 22 of Set 24 | kaspiyskiy
Processing Record 23 of Set 24 | boissevain
Processing Record 24 of Set 24 | cabedelo
Processing Record 25 of Set 24 | ijaki
City not found. Skipping...
Processing Record 26 of Set 24 | yulara
Processing Record 27 of Set 

In [35]:
len(city_data)

1100

In [28]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Weather description,Rain inches(last 3 hrs),Snow inches(last 3 hrs)
0,Champasak,14.75,106.00,70.92,39,0,3.04,LA,2020-02-23 15:38:15,clear sky,0.00,0.0
1,Tessalit,20.20,1.01,92.82,7,0,9.04,ML,2020-02-23 15:38:15,clear sky,0.00,0.0
2,Saint-Philippe,-21.36,55.77,80.60,69,75,4.70,RE,2020-02-23 15:38:15,broken clouds,0.00,0.0
3,Ushuaia,-54.80,-68.30,50.00,76,75,4.76,AR,2020-02-23 15:36:57,drizzle,0.00,0.0
4,Luderitz,-26.65,15.16,66.22,79,0,22.62,NA,2020-02-23 15:38:16,clear sky,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
545,Batemans Bay,-35.72,150.18,68.00,88,96,1.01,AU,2020-02-23 15:40:35,overcast clouds,0.00,0.0
546,Vao,-22.67,167.48,78.10,74,100,13.33,NC,2020-02-23 15:40:35,light rain,0.13,0.0
547,Touho,-20.78,165.23,70.47,90,85,4.03,NC,2020-02-23 15:40:36,light rain,0.19,0.0
548,Lufilufi,-13.87,-171.60,79.32,87,100,18.59,WS,2020-02-23 15:40:36,light rain,1.44,0.0


In [32]:
rain_count=city_data_df.loc[city_data_df["Rain inches(last 3 hrs)"]>0, ["Rain inches(last 3 hrs)"]].count()
snow_count=city_data_df.loc[city_data_df["Snow inches(last 3 hrs)"]>0, ["Snow inches(last 3 hrs)"]].count()
print(f'rain count:{rain_count},snow count:{snow_count}')

rain count:Rain inches(last 3 hrs)    61
dtype: int64,snow count:Snow inches(last 3 hrs)    27
dtype: int64


In [33]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_challenge.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")

End Program